In [ ]:
import numpy as np
import pandas as pd

In [ ]:
static = pd.read_csv('/content/static.csv')
text = pd.read_csv('/content/notes.csv')

static = static[['id', 'los_icu']]
text = text[['id', 'charttime', 'text']]

In [ ]:
data = static.set_index('id').join(text.set_index('id'))
data = data.reset_index()

In [ ]:
import re
import nltk
from nltk.tokenize import TreebankWordTokenizer

# Data Preprocessing
data['text'] = data['text'].map(lambda x: x.lower())
data['text'] =  data['text'].apply(lambda x: re.sub(r'[\r\n]+', ' ', x))
data['text'] =  data['text'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', ' ', x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature Extraction
vectorizer = TfidfVectorizer(max_features=100)
TFidf_features = vectorizer.fit_transform(data['text']).toarray()

In [ ]:
# Time series data transformation
grouped = data.groupby('id')

# Initialize lists to store sequences and subjects
sequences = []
subjects = []
#output = []

for subject, group in grouped:
    # Extract timepoints for the subject
    timepoints = group['charttime'].values
    los = group['los_icu'].values

    # Append the subject to the list of subjects
    subjects.append(subject)

    #output.append(los)

    # Retrieve feature values for each timepoint
    feature_values_sequence = [TFidf_features[i - 1] for i in range(len(timepoints))]

    # Append feature values as a sequence
    sequences.append(feature_values_sequence)
